In [1]:
from learner import Learner

import os

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import librosa
import numpy as np
import torch
from PIL import Image
from sklearn.utils import shuffle
from tqdm import tqdm
from copy import deepcopy
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

2023-08-24 10:28:05,983 - butterfly_op.py[line:23] - WARNING: Import universal_cuda fail
2023-08-24 10:28:05,984 - butterfly_op.py[line:27] - WARNING: Import hadamard_cuda fail
2023-08-24 10:28:05,985 - matrix_parametrization.py[line:23] - WARNING: Cannot import matrix_parametrization_cuda. Decomposers can only work on CPU mode


In [2]:
config = [
    ('conv2d', [32, 1, 7, 7, 1, 3]),
    ('relu', [True]),
    ('bn', [32]),
    ('conv2d', [16, 32, 5, 5, 1, 2]),
    ('relu', [True]),
    ('bn', [16]),
    ('conv2d', [8, 16, 3, 3, 1, 1]),
    ('relu', [True]),
    ('bn', [8]),
    ('flatten', []),
    ('linear', [5, 8 * 28 * 28])
]

In [81]:
from birdCallNShot import BirdCallNShot

db_train = BirdCallNShot('birdCall',
                       batchsz=32,
                       n_way=5,
                       k_shot=10,
                       k_query=15,
                       SPEC_SHAPE = (28,28))

DB: train (99, 100, 28, 28, 1) test (33, 100, 28, 28, 1)


In [82]:
x_spt, y_spt, x_qry, y_qry = map(lambda x: torch.from_numpy(x).to(torch.device('cpu')), db_train.next())
device = torch.device('cpu')

In [83]:
x_spt.shape

torch.Size([32, 50, 1, 28, 28])

In [101]:
from torchonn.op.mzi_op import project_matrix_to_unitary

def check_accuracy(data, labels, model, idx = 0):
    # print('Checking accuracy on set')   
    num_correct = 0
    num_samples = 0
    num_data_points = data.shape[0]
    
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        x, y = data[idx], labels[idx]
        x = x.to(device=torch.device("cpu"), dtype=torch.float32)  # move to device, e.g. GPU
        y = y.to(device=torch.device("cpu"), dtype=torch.long)
        scores = model(x)
        _, preds = scores.max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        # print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc
        
def train(data, labels, model, optimizer, epochs=50, idx = 0):
    model = model.to(device=torch.device("cpu"))  # move the model parameters to CPU/GPU    
    test_accuracies = [check_accuracy(x_qry, y_qry, model, idx)]
    for e in range(epochs):
        # print(f"Epoch number: {e}")
        x, y = data[idx], labels[idx]
        model.train()  # put model to training mode
        x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
        y = y.to(device=device, dtype=torch.long)

        scores = model(x)
        loss = F.cross_entropy(scores, y)

        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad()

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()

        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        
        
        optimizer.step()
        model_idx = 0
        for name, param in model.config:
            if name == 'conv2d' or name == 'linear':
                # project U to unitary
                with torch.no_grad():
                    model.parameters()[model_idx].copy_(project_matrix_to_unitary(model.parameters()[model_idx]))
                    model.parameters()[model_idx+1].copy_(project_matrix_to_unitary(model.parameters()[model_idx+1]))
                model_idx += 3
            elif name == 'bn':
                model_idx += 2

        param_idx = 0
        
        test_accuracies.append(check_accuracy(x_qry, y_qry, model, idx))
    return test_accuracies

In [112]:
lr = 0.4

models = []
models = [Learner(config, 1, 28, mb = 16) for _ in range(11)]
test_accs = []
for idx in range(1):
    models[idx].load_state_dict(torch.load('onn_birdcall_lr0.4mlr0.01shot1.pth'))
    print(f"On iteration {idx}")
    models[idx].eval()
    optimizer = optim.SGD(models[idx].parameters(), lr=lr, momentum=0.9, nesterov=True)
    test_accs.append(train(x_spt, y_spt, models[idx], optimizer, epochs = 50, idx = idx))

avg_test_accuracies = np.average(np.array(test_accs), axis = 0)

On iteration 0


In [113]:
avg_test_accuracies

array([0.16      , 0.17333333, 0.24      , 0.33333333, 0.48      ,
       0.42666667, 0.53333333, 0.57333333, 0.54666667, 0.54666667,
       0.50666667, 0.48      , 0.46666667, 0.57333333, 0.61333333,
       0.57333333, 0.66666667, 0.66666667, 0.69333333, 0.65333333,
       0.66666667, 0.66666667, 0.68      , 0.66666667, 0.68      ,
       0.69333333, 0.72      , 0.72      , 0.72      , 0.72      ,
       0.72      , 0.72      , 0.72      , 0.72      , 0.72      ,
       0.72      , 0.72      , 0.72      , 0.72      , 0.73333333,
       0.73333333, 0.73333333, 0.73333333, 0.73333333, 0.73333333,
       0.73333333, 0.73333333, 0.73333333, 0.73333333, 0.73333333,
       0.73333333])

In [114]:
torch.allclose(model.parameters()[0][0][0] @ model.parameters()[0][0][0].T, torch.eye(16), atol = 1e-6)

True

In [117]:
model.vars[7]

Parameter containing:
tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
   